In [ ]:
import SimpleITK as sitk
import numpy as np
import os

# Folder name where registration transforms are saved
folder = "TestDenoising5"

path = "./oai_zib_mri_train/train"
labelPath = "oai_zib_labelmaps/labelmaps/train"

P = [3, 3, 3]
N = [5, 5, 5]

In [ ]:
# Read boundaries
file1 = open(f"./{folder}/boundaries.txt", "r")
temp = file1.read().split("\n")[:-1]
minx = int(temp[0])
maxx = int(temp[1])
miny = int(temp[2])
maxy = int(temp[3])
minz = int(temp[4])
maxz = int(temp[5])

In [ ]:
file = open(f"./{folder}/names.txt", "r") 
name = file.read().split("\n")[0]

pathName = os.path.join(path, name)
images = []
images.append(sitk.GetArrayFromImage(sitk.ReadImage(pathName)))

shape = [[minx - (P[0]//2 + N[0]//2), maxx + (P[0]//2 + N[0]//2)],
         [miny - (P[1]//2 + N[1]//2), maxy + (P[1]//2 + N[1]//2)],
         [minz - (P[2]//2 + N[2]//2), maxz + (P[2]//2 + N[2]//2)],
        ]

copyShape = [[],[],[]]

# x range
if shape[0][0] < 0: 
    copyShape[0].append(0)
else:
    copyShape[0].append(shape[0][0])
    
if shape[0][1] > images[0].shape[0] - 1: 
    copyShape[0].append(images[0].shape[0] - 1)
else:
    copyShape[0].append(shape[0][1])

# y range
if shape[1][0] < 0: 
    copyShape[1].append(0)
else:
    copyShape[1].append(shape[1][0])
    
if shape[1][1] > images[0].shape[1] - 1: 
    copyShape[1].append(images[0].shape[1] - 1)
else:
    copyShape[1].append(shape[1][1])
    
#z range
if shape[2][0] < 0: 
    copyShape[2].append(0)
else:
    copyShape[2].append(shape[2][0])
    
if shape[2][1] > images[0].shape[2] - 1: 
    copyShape[2].append(images[0].shape[2] - 1)
else:
    copyShape[2].append(shape[2][1])
    
offset = []
offset.append(copyShape[0][0] - shape[0][0])
offset.append(copyShape[1][0] - shape[1][0])
offset.append(copyShape[2][0] - shape[2][0])

length = []
length.append(copyShape[0][1] - copyShape[0][0] + 1)
length.append(copyShape[1][1] - copyShape[1][0] + 1)
length.append(copyShape[2][1] - copyShape[2][0] + 1)

del images[0]
print(images)

In [ ]:
"""Read images and labels and make them numpy images with the proper size."""
from utils import *

images = []
labels = []

file = open(f"./{folder}/names.txt", "r") 
imageNames = file.read().split("\n")
# del imageNames[1]

for i, name in enumerate(imageNames[:2]):
    print(f'Reading {name}. {i+1}/{len(imageNames)}')
    
    pathName = os.path.join(path, name)
    
    if i >= 1:
        t = sitk.ReadTransform(f"./{folder}/{name}.tfm")
        
        image = sitk.GetArrayFromImage(resampleImage(sitk.HistogramMatching(sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                                                                               timeStep = 0.04, 
                                                                                               numberOfIterations = 10), 
                                                                            images[0]), images[0], t))

        newImage = np.zeros((shape[0][1] - shape[0][0] + 1, shape[1][1] - shape[1][0] + 1, shape[2][1] - shape[2][0] + 1), dtype="uint16", order="F")
        newImage[offset[0]:offset[0]+length[0], 
                 offset[1]:offset[1]+length[1], 
                 offset[2]:offset[2]+length[2]] = image[copyShape[0][0]:copyShape[0][1]+1,
                                                        copyShape[1][0]:copyShape[1][1]+1,
                                                        copyShape[2][0]:copyShape[2][1]+1,]
        images.append(newImage)
        
        label = sitk.GetArrayFromImage(resampleSegmentation(sitk.ReadImage(os.path.join(labelPath, name)), images[0], t))
        
        newLabels = np.zeros((shape[0][1] - shape[0][0] + 1, shape[1][1] - shape[1][0] + 1, shape[2][1] - shape[2][0] + 1,), dtype="uint8", order="F")
        newLabels[offset[0]:offset[0]+length[0], 
                  offset[1]:offset[1]+length[1], 
                  offset[2]:offset[2]+length[2]] = label[copyShape[0][0]:copyShape[0][1]+1,
                                                         copyShape[1][0]:copyShape[1][1]+1,
                                                         copyShape[2][0]:copyShape[2][1]+1,]
        labels.append(newLabels)
    else:
        images.append(sitk.CurvatureFlow(sitk.ReadImage(pathName), timeStep = 0.04, numberOfIterations = 10))

        labels.append(sitk.ReadImage(os.path.join(labelPath, name)))
        
images[0] = sitk.GetArrayFromImage(images[0])
newImage = np.zeros((shape[0][1] - shape[0][0] + 1, shape[1][1] - shape[1][0] + 1, shape[2][1] - shape[2][0] + 1), dtype="uint16", order="F")
newImage[offset[0]:offset[0]+length[0], 
         offset[1]:offset[1]+length[1], 
         offset[2]:offset[2]+length[2]] = images[0][copyShape[0][0]:copyShape[0][1]+1,
                                                    copyShape[1][0]:copyShape[1][1]+1,
                                                    copyShape[2][0]:copyShape[2][1]+1,]
images[0] = newImage

labels[0] = sitk.GetArrayFromImage(labels[0])
newLabels = np.zeros((shape[0][1] - shape[0][0] + 1, shape[1][1] - shape[1][0] + 1, shape[2][1] - shape[2][0] + 1,), dtype="uint8", order="F")
newLabels[offset[0]:offset[0]+length[0], 
          offset[1]:offset[1]+length[1], 
          offset[2]:offset[2]+length[2]] = labels[0][copyShape[0][0]:copyShape[0][1]+1,
                                                     copyShape[1][0]:copyShape[1][1]+1,
                                                     copyShape[2][0]:copyShape[2][1]+1,]
labels[0] = newLabels

In [ ]:
# Make to the proper type
images = np.array(images, order='C')
labels = np.array(labels, order='C')
P = np.array(P, dtype=np.int32)
N = np.array(N, dtype=np.int32)

In [ ]:
from cGen import cGen

segmentation = cGen.applySPBM(images[0], images[1:], labels[1:], 5, P, N, lassoTol=0.00001, lassoMaxIter=1e4, verboseX=True, 
                 verboseY=False, xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=-1, zmax=-1)

In [ ]:
# Save segmentation
originalImage = sitk.ReadImage(os.path.join(path, imageNames[0]))
originalImageShape = originalImage.GetSize()[::-1]
finalSegmentation = np.zeros(originalImageShape, dtype='uint8')
finalSegmentation[copyShape[0][0]:copyShape[0][1]+1,
                  copyShape[2][0]:copyShape[1][1]+1,
                  copyShape[2][0]:copyShape[2][1]+1,] = segmentation[offset[0]:offset[0]+length[0], 
                                                                     offset[1]:offset[1]+length[1], 
                                                                     offset[2]:offset[2]+length[2]]

finalSegmentation = sitk.GetImageFromArray(finalSegmentation)
finalSegmentation.CopyInformation(originalImage)
sitk.WriteImage(finalSegmentation, imageNames[0]+'TESTTTTTTTTTTimport.mhd')

In [ ]:
dice = Dice(labels[0], segmentation, 5)
for i, d in enumerate(dice):
    print(f'Label {i+1}: {d}')